# Creating a **combined quantification pipeline & batch processing** - part 2.5
--------------------
Now that all the function to quantify features of organelle composition, morphology, interactions, and distribution have been created, we will combine them into a wrapper function for easy use. This will then be used as part of a batch processing function allowing the the wrapper to be applied to multiple images in the same directory.

## **OBJECTIVE**
### <input type="checkbox"/> Create a **combined quantification pipeline** for **batch processing** all organelles of interest
In this notebook, the logic for quantifying the **composition**, **morphology**, **interaction**, and **distribution** of **organelles** (as many as you would like) from single cells. To do this, the quantification functions from previous notebooks (notebooks 2.1-2.4) at combined into a single function to ***process multiple organelles*** at a time and a framework for ***batch processing*** multiple cells/images is also established here.

---------
## **Combined Measurements and Batch Processing**

### summary of steps

#### **PART 1️⃣: Combined Measurements**

- **`0`** - Establish Parameters for prototype `_make_all_metrics_tables` function *(preliminary step)*

- **`1`** - Stack intensity images with observed organelles

- **`2`** - Collect region morphology data using `get_region_morphology` function

    - identify example cell region *(cellmask)*
    - run `get_region_morphology_3D`

- **`3`** - Collect organelle morphology data using `get_org_morphology_3D` function

    - identify example organelle *(lysosome)*
    - obtain example organelle segmentation
    - run `get_org_morphology_3D`

- **`4`** - Collect distribution metrics data using `get_XY_distribution` and `get_Z_distribution` functions

    - establish containers and centering object
    - obtain XY Distribution data
    - obtain Z Distribution data
    - add distribution data (from both functions) to containers

- **`5`** - Collect organelle interaction metrics using `get_contact_metrics_3D`

    - create all 2-way organelle combinations
    - identify example interaction
    - retrieve organelle segmentations involved in interaction
    - run `get_contact_metrics_3D`

- **`6`** - Combine all tables into four inter-organelle tables

- Define `_get_org_morphology_3D` function

- Run `_get_org_morphology_3D` function

- Compare to finalized `get_org_morphology_3D` function

#### **PART 2️⃣: Batch Process Quantification**

- **`0`** - Establish the image data paths *(preliminary step)*

- **`1`** - Locate the matching raw and segmentation files *(`_find_segmentation_tiff_files`)*

    - identify protoype image and segmentations to collect
    - locate folders for raw image and segmentations
    - collect all listed region and segmentation filenames
    - define `_find_segmentation_tiff_files` function
    - run `_find_segmentation_tiff_files` function
    - compare to finalized `find_segmentation_tiff_files` function

- **`2`** - Establish the parameters for prototype `_batch_process_quantification` function

- **`3`** - List raw image files and their corresponding segmentations and to collect

- **`4`** - Obtain segmentations and raw image files

    - run `find_segmentation_tiff_files` to collect segmentation filenames
    - read in linearly unmixed image file *(raw)*
    - collect the listed organelle channel intensities from the raw image file
    - identify the scale of the raw image from the metadata
    - read in organelle and region segmentations

- **`5`** - Run `make_all_metrics_tables` and store output

    - run `make_all_metrics_tables` for the prototype image
    - repeat steps **`3`** and **`4`** and run `make_all_metrics_tables` for all images in `img_file_list`

- **`6`** - Combine all per image tables into four comprehensive tables

    - batch organelle morphology table
    - batch organelle interactions table
    - batch distribution measurments table
    - batch cell region morphology table

- **`7`** - Export comprehensive tables as .csv files

- Define `_batch_process_quantification` function

- Run `_batch_process_quantification` function

## **IMPORTS**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block loads all of the necessary python packages and functions you will need for this notebook.

In [4]:
from pathlib import Path
from typing import Dict
import os
import itertools
import shutil

from infer_subc.core.file_io import (read_czi_image,
                                     import_inferred_organelle,
                                     list_image_files,
                                     export_ome_tiff,
                                     sample_input_quant,
                                     sample_input_batch)
from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats_helpers import *
from infer_subc.organelles import *

from datetime import datetime

import time
%load_ext autoreload
%autoreload 2

## **LOAD AND READ IN IMAGE FOR PROCESSING**
> ###### 📝 **Specifically, this will include the raw image and the outputs from segmentation**

#### &#x1F6D1; &#x270D; **User Input Required:**

In [ ]:
### USER INPUT REQUIRED ###
# If using the sample data, select which cell type you would like analyze ("neuron" or "astrocyte"):
# If not using the sample data, set cell_type to None
sample_data_type = "neuron"

# Specify which file you'd like to segment from the img_file_list
test_img_n = 0

# If you are not using the sample data, please edit "USER SPECIFIED" as necessary.
## Define the path to the directory that contains the input image folder.
data_root_path = Path("USER SPECIFIED")

# Specify the file type of your raw data that will be analyzed. Ex) ".czi" or ".tiff"
raw_img_type = "USER SPECIFIED"

## Specify which subfolder that contains the input data and the input data file extension
in_data_path = data_root_path / "USER SPECIFIED"

## Specify which subfolder contains the segmentation outputs and their file type
seg_data_path = data_root_path / "USER SPECIFIED"
seg_img_type = "USER SPECIFIED"

## Specify the output folder to save the segmentation outputs if.
## If its not already created, the code below will creat it for you
out_data_path = data_root_path / "USER SPECIFIED"

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
if sample_data_type != None:
    # All of the following options are correctly set to work with the sample data;
    data_root_path, raw_img_type, seg_img_type, in_data_path, seg_data_path, out_data_path = sample_input_quant(sample_data_type)

In [3]:
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

raw_file_list = list_image_files(in_data_path, raw_img_type)
seg_file_list = list_image_files(seg_data_path, seg_img_type)
# pd.set_option('display.max_colwidth', None)
# pd.DataFrame({"Image Name":img_file_list})

making c:\Users\redre\Documents\CohenLab\MSI-3D-analysis\infer-subc\sample_data\example_astrocyte\quant


In [4]:
raw_img_name = raw_file_list[test_img_n]

raw_img_data, raw_meta_dict = read_czi_image(raw_img_name)

channel_names = raw_meta_dict['name']
img = raw_meta_dict['metadata']['aicsimage']
scale = raw_meta_dict['scale']
channel_axis = raw_meta_dict['channel_axis']

In [5]:
## For each import, change the string to match the suffix on the segmentation files (i.e., the stuff following the "-")

# masks
masks_seg_names = ['masks','masks_A', 'masks_B']
for m in masks_seg_names:
    if m in [i.stem.split("-")[-1] for i in seg_file_list]:
        mask_seg = import_inferred_organelle(m, raw_meta_dict, seg_data_path, seg_img_type)
        nuc_seg, cell_seg, cyto_seg = mask_seg
        break

if 'nuc' in [i.stem.split("-")[-1] for i in seg_file_list]:
    nuc_seg = import_inferred_organelle("nuc", raw_meta_dict, seg_data_path, seg_img_type)
    cell_seg = import_inferred_organelle("cell", raw_meta_dict, seg_data_path, seg_img_type)
    cyto_seg = import_inferred_organelle("cyto", raw_meta_dict, seg_data_path, seg_img_type)

#organelles
lyso_seg = import_inferred_organelle("lyso", raw_meta_dict, seg_data_path, seg_img_type)
mito_seg = import_inferred_organelle("mito", raw_meta_dict, seg_data_path, seg_img_type)
golgi_seg = import_inferred_organelle("golgi", raw_meta_dict, seg_data_path, seg_img_type)
perox_seg = import_inferred_organelle("perox", raw_meta_dict, seg_data_path, seg_img_type)
ER_seg = import_inferred_organelle("ER", raw_meta_dict, seg_data_path, seg_img_type)
LD_seg = import_inferred_organelle("LD", raw_meta_dict, seg_data_path, seg_img_type)

loaded  inferred 4D `masks_B`  from c:\Users\redre\Documents\CohenLab\MSI-3D-analysis\infer-subc\sample_data\example_astrocyte\seg 
loaded  inferred 3D `lyso`  from c:\Users\redre\Documents\CohenLab\MSI-3D-analysis\infer-subc\sample_data\example_astrocyte\seg 
loaded  inferred 3D `mito`  from c:\Users\redre\Documents\CohenLab\MSI-3D-analysis\infer-subc\sample_data\example_astrocyte\seg 
loaded  inferred 3D `golgi`  from c:\Users\redre\Documents\CohenLab\MSI-3D-analysis\infer-subc\sample_data\example_astrocyte\seg 
loaded  inferred 3D `perox`  from c:\Users\redre\Documents\CohenLab\MSI-3D-analysis\infer-subc\sample_data\example_astrocyte\seg 
loaded  inferred 3D `ER`  from c:\Users\redre\Documents\CohenLab\MSI-3D-analysis\infer-subc\sample_data\example_astrocyte\seg 
loaded  inferred 3D `LD`  from c:\Users\redre\Documents\CohenLab\MSI-3D-analysis\infer-subc\sample_data\example_astrocyte\seg 


# ***PART 1️⃣: Combined Measurements***

## **`0` - Establish Parameters for prototype `_make_all_metrics_tables` function *(preliminary step)***

> ###### **📝 some variable names will differ in the defined prototype function to make a distinction between global and local variables**

In [6]:
# names of organelles we have only two for this example
organelle_names = ["lyso", "ER"]

# to get the intensities of the test organelles
organelle_channels = [3,1]

# create intensities from raw as list
intensities = [raw_img_data[ch] for ch in organelle_channels]

# load organelles as list
organelles = [lyso_seg, ER_seg]

# load regions as list (only cellmask for this example)
regions = [cell_seg]

# list of region names
region_names = ['cell']

# Number of bins to be used during calculation of distribution metrics
dist_num_bins=5

# If set to true the bins will be distributed from the center of the centering object to the edge of the cellmask
# If set to false the bins will be distributed from the edge of the centering object to the edge of the cellmask
dist_center_on=False

# Whether or not to include the centering object as the first bin
dist_keep_center_as_bin=True

# the number of zernike degrees to include for the zernike shape descriptors
dist_zernike_degrees=9

# Whether or not to include distribution data for the interaction sites
include_contact_dist=True

# select the cellmask as the masking object for the organelle segmentations
# This is done differently in the function, but is simplified here
mask = cell_seg

## **`1` - Stack intensity images with observed organelles**

In [7]:
# create np.ndarray of intensity images
raw_image = np.stack(intensities)

## **`2` - Collect region morphology data using `get_region_morphology` function**

> ###### **📝 In the following cells, the cellmask will serve as the example region. The finalized prototype function will collect the morphology data for all regions listed as input.**

- identify example region *(cellmask)*

In [8]:
# contains the per region morphological information
region_tabs = []

# Establishing the cellmask as the example region
r_name = 'cell'
r = region_names.index(r_name)

- run `get_region_morphology_3D`

In [9]:
region = regions[r]
region_metrics = get_region_morphology_3D(region_seg=region, 
                                            region_name=r_name,
                                            channel_names=organelle_names,
                                            intensity_img=raw_image, 
                                            mask=mask,
                                            scale=scale)
region_tabs.append(region_metrics)

# Show cellmask morphology output
region_tabs[0]

,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,...,solidity,axis_major_length,min_intensity-lyso_ch,min_intensity-ER_ch,max_intensity-lyso_ch,max_intensity-ER_ch,mean_intensity-lyso_ch,mean_intensity-ER_ch,standard_deviation_intensity-lyso_ch,standard_deviation_intensity-ER_ch
0,cell,1,"(0.3961, 0.0799, 0.0799)",3.42331,57.482342,73.509037,2,288,610,18,...,0.300887,72.743973,0.013644,0.025324,63768.566406,184831.34375,2148.092285,10064.098633,4296.401855,11873.426758


## **`3` - Collect organelle morphology data using `get_org_morphology_3D` function**

> ###### **📝 In the following cells, the lysosome will serve as an example. The finalized prototype function will collect the morphology data for all organelles listed as input.**

- identify example organelle *(lysosome)*

In [10]:
# contains the per organelle morphological information
org_tabs = []

# Establishing the lysosome as the example organelle
target = "lyso"
j = organelle_names.index(target)

- obtain example organelle segmentation

In [11]:
# organelle intensity image
org_img = intensities[j]

# organelle segmentation
if target == 'ER':
    # ensure ER is only one object
    org_obj = (organelles[j] > 0).astype(np.uint16)
else:
    org_obj = organelles[j]

- run `get_org_morphology_3D`

In [12]:
# collect the morphology data for the organelle
org_metrics = get_org_morphology_3D(segmentation_img=org_obj, 
                                    seg_name=target,
                                    intensity_img=org_img, 
                                    mask=mask,
                                    scale=scale)

# add the morphology data to the container
org_tabs.append(org_metrics)

# Show organelle morphology output
org_tabs[0]

c:\Users\redre\anaconda3\envs\infer-subc-sample\lib\site-packages\skimage\measure\_regionprops.py:430: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 274070751  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  4  Error-roundoff 5.5e-15  _one-merge 3.9e-14
  _near-inside 1.9e-13  Visible-distance 1.1e-14  U-max-coplanar 1.1e-14
  Width-outside 2.2e-14  _wide-facet 6.7e-14  _maxoutside 4.4e-14

  return convex_hull_image(self.image)
c:\Users\redre\anaconda3\envs\infer-subc-sample\lib\site-packages\skimage\measure\_regionprops.py:629: RuntimeWarning: divide by zero encountered in scalar divide
  return self.area / self.area_convex
c:\Users\redre\anaconda3\envs\infer-subc-sample\lib\site-packages\skimage\measure\_regionprops.py:430: U

,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,...,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,lyso,24,"(0.3961, 0.0799, 0.0799)",3.277839,56.855426,69.993317,2,571,711,18,...,3.653857,10.312543,0.147918,-147,0.327005,28.333220,464.521332,63768.566406,7695.379883,6348.969238
1,lyso,159,"(0.3961, 0.0799, 0.0799)",1.219543,38.550233,49.877461,2,479,621,5,...,14.795673,0.716261,0.452381,1,0.808511,1.141136,560.063293,10399.141602,3539.507812,2599.490234
2,lyso,173,"(0.3961, 0.0799, 0.0799)",1.621445,42.850571,51.407452,2,525,635,7,...,8.987179,1.314719,0.215103,1,0.643836,2.840378,1674.058472,19969.951172,5392.369141,3871.452148
3,lyso,177,"(0.3961, 0.0799, 0.0799)",1.210693,44.246893,55.917647,3,550,696,5,...,17.491580,0.635173,0.414063,1,0.757143,0.731893,1060.050659,8507.080078,4220.408691,1869.155029
4,lyso,182,"(0.3961, 0.0799, 0.0799)",1.504691,48.415260,54.300554,3,602,674,7,...,10.861088,0.944025,0.439394,1,0.820755,1.479506,2134.108154,10107.143555,4662.395020,2259.895996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,lyso,713,"(0.3961, 0.0799, 0.0799)",6.362738,59.569871,77.558796,15,742,967,18,...,11.299208,0.768852,0.489583,1,0.758065,1.161805,977.955383,9678.432617,3686.084473,2204.601318
209,lyso,714,"(0.3961, 0.0799, 0.0799)",5.941365,61.727967,79.618329,15,771,994,16,...,35.141437,0.351734,0.750000,1,inf,0.359070,2136.609863,2640.494385,2337.579102,160.849258
210,lyso,718,"(0.3961, 0.0799, 0.0799)",6.337456,57.321999,78.188166,16,716,977,17,...,32.871165,0.351734,1.000000,1,inf,0.291925,2182.252441,3363.850586,2696.972412,416.378876
211,lyso,719,"(0.3961, 0.0799, 0.0799)",6.337456,57.721734,78.987636,16,721,987,17,...,32.871165,0.351734,1.000000,1,inf,0.291925,2248.788574,3358.940674,2739.195801,310.354767


## **`4` - Collect distribution metrics data using `get_XY_distribution` and `get_Z_distribution` functions**

- establish containers and centering object

In [13]:
# contains the per organelle distribution information
dist_tabs = []

# contains the masks of the concentric ring bins per organelle
XY_bins = []

#contains the masks of the 8 radial wedges per organelle
XY_wedges = []

# Although set to the nucleus here, the protoype function allows the selection of the centering object
centering = nuc_seg

- obtain XY Distribution data

In [14]:
# collect the XY distribution data for the organelle
XY_org_distribution, XY_bin_masks, XY_wedge_masks = get_XY_distribution(mask=mask,
                                                                        centering_obj=centering,
                                                                        obj=org_obj,
                                                                        obj_name=target,
                                                                        scale=scale,
                                                                        num_bins=dist_num_bins,
                                                                        center_on=dist_center_on,
                                                                        keep_center_as_bin=dist_keep_center_as_bin,
                                                                        zernike_degrees=dist_zernike_degrees)

- obtain Z Distribution data

In [15]:
# collect the Z distribution data for the organelle
Z_org_distribution = get_Z_distribution(mask=mask, 
                                                obj=org_obj,
                                                obj_name=target,
                                                center_obj=centering,
                                                scale=scale)

- add distribution data (from both functions) to containers

In [16]:
# Combine distribution data
org_distribution_metrics = pd.merge(XY_org_distribution, Z_org_distribution,on=["object", "scale"])

# Add distribution data to the container
dist_tabs.append(org_distribution_metrics)

# Add the mask of the concentric ring bins to the bins container
XY_bins.append(XY_bin_masks)

# Add the mask of the 8 radial wedges to the wedges container
XY_wedges.append(XY_wedge_masks)

## **`5` - Collect organelle interaction metrics using `get_contact_metrics_3D`**

- create all 2-way organelle combinations (non-redundant)

> ###### **📝 This is an example of how the list of combinations is determined in the finalized prototype function. In the following steps, only the lysosome and ER interaction data will be quantified; this is performed for all combinations in the finalized prototype function.**

In [17]:
list(itertools.combinations(["lyso", "mito","golgi","perox","ER","LD"], 2))

[('lyso', 'mito'),
 ('lyso', 'golgi'),
 ('lyso', 'perox'),
 ('lyso', 'ER'),
 ('lyso', 'LD'),
 ('mito', 'golgi'),
 ('mito', 'perox'),
 ('mito', 'ER'),
 ('mito', 'LD'),
 ('golgi', 'perox'),
 ('golgi', 'ER'),
 ('golgi', 'LD'),
 ('perox', 'ER'),
 ('perox', 'LD'),
 ('ER', 'LD')]

In [18]:
contact_combos = list(itertools.combinations(organelle_names, 2))

# container to keep contact data in
contact_tabs = []

- identify example interaction

In [19]:
# Establshing lyso and ER as the only pair
pair = contact_combos[0]

# pair names
a_name = pair[0]
b_name = pair[1]

- retrieve organelle segmentations involved in interaction

In [20]:
# segmentations to measure
if a_name == 'ER':
    # ensure ER is only one object
    a = (organelles[organelle_names.index(a_name)] > 0).astype(np.uint16)
else:
    a = organelles[organelle_names.index(a_name)]

if b_name == 'ER':
    # ensure ER is only one object
    b = (organelles[organelle_names.index(b_name)] > 0).astype(np.uint16)
else:
    b = organelles[organelle_names.index(b_name)]

> ###### **📝 In the finalized prototype function, this operation will differ based on the value of the `include_contact_dist` input. It is set to true here resulting in the inclusion of the distribution data of lyso and ER interaction sites.**

- run `get_contact_metrics_3D`

In [21]:
contact_tab, contact_dist_tab = get_contact_metrics_3D(a, a_name, 
                                                       b, b_name, 
                                                       mask,
                                                       scale, 
                                                       include_dist = include_contact_dist,
                                                       dist_centering_obj=centering,
                                                       dist_num_bins=dist_num_bins,
                                                       dist_zernike_degrees=dist_zernike_degrees,
                                                       dist_center_on=dist_center_on,
                                                       dist_keep_center_as_bin=dist_keep_center_as_bin)

# Adds distribution data to containter
dist_tabs.append(contact_dist_tab)

# Adds interaction data to container
contact_tabs.append(contact_tab)

c:\Users\redre\anaconda3\envs\infer-subc-sample\lib\site-packages\skimage\measure\_regionprops.py:430: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 274658996  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  4  Error-roundoff 5.5e-15  _one-merge 3.9e-14
  _near-inside 1.9e-13  Visible-distance 1.1e-14  U-max-coplanar 1.1e-14
  Width-outside 2.2e-14  _wide-facet 6.7e-14  _maxoutside 4.4e-14

  return convex_hull_image(self.image)
c:\Users\redre\anaconda3\envs\infer-subc-sample\lib\site-packages\skimage\measure\_regionprops.py:629: RuntimeWarning: divide by zero encountered in scalar divide
  return self.area / self.area_convex
c:\Users\redre\anaconda3\envs\infer-subc-sample\lib\site-packages\skimage\measure\_regionprops.py:430: U

## **`6` - Combine all tables into four inter-organelle tables**

In [22]:
# Cell region morphology table
test_final_region_tab = pd.concat(region_tabs, ignore_index=True)
test_final_region_tab.insert(loc=0,column='image_name',value=raw_img_name.stem)

# Organelle morphology table
test_final_org_tab = pd.concat(org_tabs, ignore_index=True)
test_final_org_tab.insert(loc=0,column='image_name',value=raw_img_name.stem)

# Organelle interaction table
test_final_contact_tab = pd.concat(contact_tabs, ignore_index=True)
test_final_contact_tab.insert(loc=0,column='image_name',value=raw_img_name.stem)

# Distribution metrics table
test_combined_dist_tab = pd.concat(dist_tabs, ignore_index=True)
test_combined_dist_tab.insert(loc=0,column='image_name',value=raw_img_name.stem)

In [23]:
test_final_region_tab

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,...,solidity,axis_major_length,min_intensity-lyso_ch,min_intensity-ER_ch,max_intensity-lyso_ch,max_intensity-ER_ch,mean_intensity-lyso_ch,mean_intensity-ER_ch,standard_deviation_intensity-lyso_ch,standard_deviation_intensity-ER_ch
0,05052022_astro_control_2_Linear unmixing_0_cml...,cell,1,"(0.3961, 0.0799, 0.0799)",3.42331,57.482342,73.509037,2,288,610,...,0.300887,72.743973,0.013644,0.025324,63768.566406,184831.34375,2148.092285,10064.098633,4296.401855,11873.426758


In [24]:
test_final_org_tab

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,...,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,24,"(0.3961, 0.0799, 0.0799)",3.277839,56.855426,69.993317,2,571,711,...,3.653857,10.312543,0.147918,-147,0.327005,28.333220,464.521332,63768.566406,7695.379883,6348.969238
1,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,159,"(0.3961, 0.0799, 0.0799)",1.219543,38.550233,49.877461,2,479,621,...,14.795673,0.716261,0.452381,1,0.808511,1.141136,560.063293,10399.141602,3539.507812,2599.490234
2,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,173,"(0.3961, 0.0799, 0.0799)",1.621445,42.850571,51.407452,2,525,635,...,8.987179,1.314719,0.215103,1,0.643836,2.840378,1674.058472,19969.951172,5392.369141,3871.452148
3,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,177,"(0.3961, 0.0799, 0.0799)",1.210693,44.246893,55.917647,3,550,696,...,17.491580,0.635173,0.414063,1,0.757143,0.731893,1060.050659,8507.080078,4220.408691,1869.155029
4,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,182,"(0.3961, 0.0799, 0.0799)",1.504691,48.415260,54.300554,3,602,674,...,10.861088,0.944025,0.439394,1,0.820755,1.479506,2134.108154,10107.143555,4662.395020,2259.895996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,713,"(0.3961, 0.0799, 0.0799)",6.362738,59.569871,77.558796,15,742,967,...,11.299208,0.768852,0.489583,1,0.758065,1.161805,977.955383,9678.432617,3686.084473,2204.601318
209,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,714,"(0.3961, 0.0799, 0.0799)",5.941365,61.727967,79.618329,15,771,994,...,35.141437,0.351734,0.750000,1,inf,0.359070,2136.609863,2640.494385,2337.579102,160.849258
210,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,718,"(0.3961, 0.0799, 0.0799)",6.337456,57.321999,78.188166,16,716,977,...,32.871165,0.351734,1.000000,1,inf,0.291925,2182.252441,3363.850586,2696.972412,416.378876
211,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,719,"(0.3961, 0.0799, 0.0799)",6.337456,57.721734,78.987636,16,721,987,...,32.871165,0.351734,1.000000,1,inf,0.291925,2248.788574,3358.940674,2739.195801,310.354767


In [25]:
test_final_contact_tab

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,...,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length
0,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,291_1,"(0.3961, 0.0799, 0.0799)",1.523427,40.809869,48.835317,3,509,610,...,513,613,1.006807,0.032911,30.591692,0.397602,0.541667,1,0.866667,0.649579
1,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,295_1,"(0.3961, 0.0799, 0.0799)",1.615228,41.870424,49.377136,3,519,613,...,529,623,3.204565,0.194935,16.439137,0.719389,0.256667,1,0.641667,0.999857
2,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,173_1,"(0.3961, 0.0799, 0.0799)",1.862983,42.620870,51.213805,3,525,635,...,545,650,9.093765,0.665817,13.658048,1.083392,0.219167,1,0.552521,1.992485
3,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,24_1,"(0.3961, 0.0799, 0.0799)",3.406541,56.299612,69.536981,3,573,713,...,842,1046,2419.505497,304.637990,7.942232,8.348244,0.095953,-553,0.210602,27.040554
4,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,24_1,"(0.3961, 0.0799, 0.0799)",1.188273,50.753020,62.811693,3,633,784,...,638,788,1.007197,0.030379,33.153850,0.387133,0.600000,1,inf,0.517616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,719_1,"(0.3961, 0.0799, 0.0799)",6.337456,57.721734,78.987636,16,721,987,...,724,990,0.748957,0.022785,32.871165,0.351734,1.000000,1,inf,0.291925
304,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,720_1,"(0.3961, 0.0799, 0.0799)",6.375179,57.992031,77.632344,16,723,968,...,729,975,1.638831,0.053164,30.825878,0.466523,0.250000,1,0.583333,0.777529
305,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,710_1,"(0.3961, 0.0799, 0.0799)",6.337456,58.521204,75.709809,16,732,947,...,733,948,0.180947,0.002532,71.474577,0.169096,1.000000,1,inf,0.000000
306,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,713_1,"(0.3961, 0.0799, 0.0799)",6.348161,59.586444,77.544269,16,742,967,...,750,975,2.711128,0.093670,28.943365,0.563466,0.289062,1,0.521127,0.976748


In [26]:
test_combined_dist_tab

,image_name,object,scale,XY_n_bins,XY_bins,XY_mask_vox_cnt_perbin,XY_obj_vox_cnt_perbin,XY_center_vox_cnt_perbin,XY_n_pix_perbin,XY_portion_pix_perbin,...,zernike_center_phs,Z_n_slices,Z_slices,Z_mask_vox_cnt,Z_obj_vox_cnt,Z_center_vox_cnt,Z_height,Z_mask_volume,Z_obj_volume,Z_center_volume
0,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,"(0.3961, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[217355, 95389, 135234, 192049, 383969]","[16618, 42894, 57510, 66988, 66838]","[79970, 0, 0, 0, 0]","[15534.0, 7146.0, 10646.0, 17048.0, 68991.0]","[0.13013865035814517, 0.05986679512419889, 0.0...",...,"[1.5707963267948966, -2.7370885998619583, -1.5...",18,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 7572, 64134, 76764, 81590, 89670, 98309...","[0, 0, 849, 9022, 22719, 26531, 25925, 26496, ...","[0, 0, 0, 0, 0, 0, 0, 9607, 13937, 14701, 1455...",7.129638,"[0.0, 0.0, 19.1694619326348, 162.3632160047015...","[0.0, 0.0, 2.149349337137737, 22.8403176909972...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.3213181..."
1,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,"(0.3961, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[217355, 95389, 135234, 192049, 383969]","[9082, 20423, 33707, 40647, 28790]","[79970, 0, 0, 0, 0]","[15534.0, 7146.0, 10646.0, 17048.0, 68991.0]","[0.13013865035814517, 0.05986679512419889, 0.0...",...,"[1.5707963267948966, -2.7370885998619583, -1.5...",18,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 7572, 64134, 76764, 81590, 89670, 98309...","[0, 0, 0, 134, 6871, 16216, 16423, 15835, 1520...","[0, 0, 0, 0, 0, 0, 0, 9607, 13937, 14701, 1455...",7.129638,"[0.0, 0.0, 19.1694619326348, 162.3632160047015...","[0.0, 0.0, 0.0, 0.339237704565909, 17.39479304...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.3213181..."


## **Define `_make_all_metrics_tables` function**

In [27]:
# organelle_to_colname = {"nuc":"NU", "lyso": "LY", "mito":"MT", "golgi":"GL", "perox":"PR", "ER":"ER", "LD":"LD", "cell":"CM", "cyto":"CY", "nucleus": "N1","nuclei":"NU",}

def _make_all_metrics_tables(source_file: str,
                             list_obj_names: List[str],
                             list_obj_segs: List[np.ndarray],
                             list_intensity_img: List[np.ndarray],
                             list_region_names: List[str],
                             list_region_segs: List[np.ndarray],
                             mask: str,
                             dist_centering_obj:str, 
                             dist_num_bins: int,
                             dist_center_on: bool=False,
                             dist_keep_center_as_bin: bool=True,
                             dist_zernike_degrees: Union[int, None]=None,
                             scale: Union[tuple,None] = None,
                             include_contact_dist:bool=True):
    """
    Measure the composition, morphology, distribution, and contacts of multiple organelles in a cell

    Parameters:
    ----------
    source_file: str
        file path; this is used for recorder keeping of the file name in the output data tables
    list_obj_names: List[str]
        a list of object names (strings) that will be measured; this should match the order in list_obj_segs
    list_obj_segs: List[np.ndarray]
        a list of 3D (ZYX) segmentation np.ndarrays that will be measured per cell; the order should match the list_obj_names 
    list_intensity_img: List[np.ndarray]
        a list of 3D (ZYX) grayscale np.ndarrays that will be used to measure fluoresence intensity in each region and object
    list_region_names: List[str]
        a list of region names (strings); these should include the mask (entire region being measured - usually the cell) 
        and other sub-mask regions from which we can meausure the objects in (ex - nucleus, neurites, soma, etc.). It should 
        also include the centering object used when created the XY distribution bins.
        The order should match the list_region_segs
    list_region_segs: List[np.ndarray]
        a list of 3D (ZYX) binary np.ndarrays of the region masks; the order should match the list_region_names.
    mask: str
        a str of which region name (contained in the list_region_names list) should be used as the main mask (e.g., cell mask)
    dist_centering_obj:str
        a str of which region name (contained in the list_region_names list) should be used as the centering object in 
        get_XY_distribution()
    dist_num_bins: int
        the number of concentric rings to draw between the centering object and edge of the mask in get_XY_distribution()
    dist_center_on: bool=False,
        for get_XY_distribution:
        True = distribute the bins from the center of the centering object
        False = distribute the bins from the edge of the centering object
    dist_keep_center_as_bin: bool=True
        for get_XY_distribution:
        True = include the centering object area when creating the bins
        False = do not include the centering object area when creating the bins
    dist_zernike_degrees: Union[int, None]=None
        for get_XY_distribution:
        the number of zernike degrees to include for the zernike shape descriptors; if None, the zernike measurements will not 
        be included in the output
    scale: Union[tuple,None] = None
        a tuple that contains the real world dimensions for each dimension in the image (Z, Y, X)
    include_contact_dist:bool=True
        whether to include the distribution of contact sites in get_contact_metrics_3d(); True = include contact distribution

    Returns:
    ----------
    4 Dataframes of measurements of organelle morphology, region morphology, contact morphology, and organelle/contact distributions

    """
    start = time.time()
    count = 0

    # segmentation image for all masking steps below
    mask = list_region_segs[list_region_names.index(mask)]

    ######################
    # measure cell regions
    ######################
    # create np.ndarray of intensity images
    raw_image = np.stack(list_intensity_img)
    
    # container for region data
    region_tabs = []
    for r, r_name in enumerate(list_region_names):
        region = list_region_segs[r]
        region_metrics = get_region_morphology_3D(region_seg=region, 
                                                  region_name=r_name,
                                                  channel_names=list_obj_names,
                                                  intensity_img=raw_image, 
                                                  mask=mask,
                                                  scale=scale)
        region_tabs.append(region_metrics)

    ##############################################################
    # loop through all organelles to collect measurements for each
    ##############################################################
    # containers to collect per organelle information
    org_tabs = []
    dist_tabs = []
    XY_bins = []
    XY_wedges = []

    for j, target in enumerate(list_obj_names):
        # organelle intensity image
        org_img = list_intensity_img[j]

        # organelle segmentation
        if target == 'ER':
            # ensure ER is only one object
            org_obj = (list_obj_segs[j] > 0).astype(np.uint16)
        else:
            org_obj = list_obj_segs[j]

        ##########################################################
        # measure organelle morphology & number of objs contacting
        ##########################################################
        org_metrics = get_org_morphology_3D(segmentation_img=org_obj, 
                                            seg_name=target,
                                            intensity_img=org_img, 
                                            mask=mask,
                                            scale=scale)

        org_tabs.append(org_metrics)

        ################################
        # measure organelle distribution 
        ################################
        centering = list_region_segs[list_region_names.index(dist_centering_obj)]
        XY_org_distribution, XY_bin_masks, XY_wedge_masks = get_XY_distribution(mask=mask,
                                                                                centering_obj=centering,
                                                                                obj=org_obj,
                                                                                obj_name=target,
                                                                                scale=scale,
                                                                                num_bins=dist_num_bins,
                                                                                center_on=dist_center_on,
                                                                                keep_center_as_bin=dist_keep_center_as_bin,
                                                                                zernike_degrees=dist_zernike_degrees)
        Z_org_distribution = get_Z_distribution(mask=mask, 
                                                obj=org_obj,
                                                obj_name=target,
                                                center_obj=centering,
                                                scale=scale)
        
        org_distribution_metrics = pd.merge(XY_org_distribution, Z_org_distribution,on=["object", "scale"])

        dist_tabs.append(org_distribution_metrics)
        XY_bins.append(XY_bin_masks)
        XY_wedges.append(XY_wedge_masks)

    #######################################
    # collect non-redundant contact metrics 
    #######################################
    # list the non-redundant organelle pairs
    contact_combos = list(itertools.combinations(list_obj_names, 2))

    # container to keep contact data in
    contact_tabs = []

    # loop through each pair and measure contacts
    for pair in contact_combos:
        # pair names
        a_name = pair[0]
        b_name = pair[1]

        # segmentations to measure
        if a_name == 'ER':
            # ensure ER is only one object
            a = (list_obj_segs[list_obj_names.index(a_name)] > 0).astype(np.uint16)
        else:
            a = list_obj_segs[list_obj_names.index(a_name)]
        
        if b_name == 'ER':
            # ensure ER is only one object
            b = (list_obj_segs[list_obj_names.index(b_name)] > 0).astype(np.uint16)
        else:
            b = list_obj_segs[list_obj_names.index(b_name)]
        

        if include_contact_dist == True:
            contact_tab, contact_dist_tab = get_contact_metrics_3D(a, a_name, 
                                                                   b, b_name, 
                                                                   mask, 
                                                                   scale, 
                                                                   include_dist=include_contact_dist,
                                                                   dist_centering_obj=centering,
                                                                   dist_num_bins=dist_num_bins,
                                                                   dist_zernike_degrees=dist_zernike_degrees,
                                                                   dist_center_on=dist_center_on,
                                                                   dist_keep_center_as_bin=dist_keep_center_as_bin)
            dist_tabs.append(contact_dist_tab)
        else:
            contact_tab = get_contact_metrics_3D(a, a_name, 
                                                 b, b_name, 
                                                 mask, 
                                                 scale, 
                                                 include_dist=include_contact_dist)
        contact_tabs.append(contact_tab)


    ###########################################
    # combine all tabs into one table per type:
    ###########################################
    final_org_tab = pd.concat(org_tabs, ignore_index=True)
    final_org_tab.insert(loc=0,column='image_name',value=source_file.stem)

    final_contact_tab = pd.concat(contact_tabs, ignore_index=True)
    final_contact_tab.insert(loc=0,column='image_name',value=source_file.stem)

    combined_dist_tab = pd.concat(dist_tabs, ignore_index=True)
    combined_dist_tab.insert(loc=0,column='image_name',value=source_file.stem)

    final_region_tab = pd.concat(region_tabs, ignore_index=True)
    final_region_tab.insert(loc=0,column='image_name',value=source_file.stem)

    end = time.time()
    print(f"It took {(end-start)/60} minutes to quantify one image.")
    return final_org_tab, final_contact_tab, combined_dist_tab, final_region_tab

## **Run `_get_org_morphology_3D` function**

In [28]:
# TODO: things to fix - 
# figure out what is causing the convex hull error - I'm guessing major axis
organelle_names = ["LD","ER","golgi","lyso","mito","perox"]
organelles = [LD_seg, ER_seg, golgi_seg, lyso_seg, mito_seg, perox_seg]
organelle_channels = [0,1,2,3,4,5]
intensities = [raw_img_data[ch] for ch in organelle_channels]
regions = [cell_seg, cyto_seg, nuc_seg]
region_names = ['cell', 'cyto', 'nuc']

test_final_org_tab, test_final_contact_tab, test_combined_dist_tab, test_final_regions_tab = _make_all_metrics_tables(source_file=raw_img_name,
                                                                                                                      list_obj_names=organelle_names,
                                                                                                                      list_obj_segs=organelles,
                                                                                                                      list_intensity_img=intensities,
                                                                                                                      list_region_names=region_names,
                                                                                                                      list_region_segs=regions,
                                                                                                                      mask='cell',
                                                                                                                      dist_centering_obj='nuc',
                                                                                                                      dist_num_bins=5,
                                                                                                                      dist_center_on=False,
                                                                                                                      dist_keep_center_as_bin=True,
                                                                                                                      dist_zernike_degrees=9,
                                                                                                                      scale=scale,
                                                                                                                      include_contact_dist=True)

import warnings
warnings.simplefilter('ignore')

c:\Users\redre\anaconda3\envs\infer-subc-sample\lib\site-packages\skimage\measure\_regionprops.py:430: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 277650642  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  6  Error-roundoff 7.4e-15  _one-merge 5.2e-14
  _near-inside 2.6e-13  Visible-distance 1.5e-14  U-max-coplanar 1.5e-14
  Width-outside 3e-14  _wide-facet 8.9e-14  _maxoutside 5.9e-14

  return convex_hull_image(self.image)
c:\Users\redre\anaconda3\envs\infer-subc-sample\lib\site-packages\skimage\measure\_regionprops.py:629: RuntimeWarning: divide by zero encountered in scalar divide
  return self.area / self.area_convex
c:\Users\redre\anaconda3\envs\infer-subc-sample\lib\site-packages\skimage\measure\_regionprops.py:430: Use

It took 6.723730174700419 minutes to quantify one image.


## **Compare to finalized `get_org_morphology_3D` function**

In [29]:
from infer_subc.utils.stats_helpers import make_all_metrics_tables

official_org, official_contacts, official_dist, official_regions = make_all_metrics_tables(source_file=raw_img_name,
                                                                                           list_obj_names=organelle_names,
                                                                                           list_obj_segs=organelles,
                                                                                           list_intensity_img=intensities,
                                                                                           list_region_names=region_names,
                                                                                           list_region_segs=regions,
                                                                                           mask='cell',
                                                                                           dist_centering_obj='nuc',
                                                                                           dist_num_bins=5,
                                                                                           dist_center_on=False,
                                                                                           dist_keep_center_as_bin=True,
                                                                                           dist_zernike_degrees=9,
                                                                                           scale=scale,
                                                                                           include_contact_dist=True)

test_final_org_tab.equals(official_org), test_final_contact_tab.equals(official_contacts), test_combined_dist_tab.equals(official_dist), test_final_regions_tab.equals(official_regions)

It took 7.270024621486664 minutes to quantify one image.


(True, True, False, True)

In [30]:
test_combined_dist_tab.compare(official_dist)

zernike_obj_mag  \
                                                 self   
7   [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
10  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   

                                                       
                                                other  
7   [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...  
10  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...

# ***PART 2️⃣: Batch Process Quantification***

## **`0` - Establish the image data paths *(preliminary step)***

#### &#x1F6D1; &#x270D; **User Input Required:**

In [ ]:
# Whether or not you are using the sample data
use_sample = True

###########################################################################
# If you are not using the sample data, please edit the below as necessary.
###########################################################################

# all the imaging data goes here
data_root_path = Path("USER SPECIFIED")

# linearly unmixed ".czi" files are here
raw_data_path = data_root_path / "USER SPECIFIED"

# list of lineary unmixed ".czi" files
raw_file_list = list_image_files(raw_data_path, "USER SPECIFIED")

# adding an additional list of image paths for the matching segmentation files
seg_data_path = data_root_path / "USER SPECIFIED"
seg_file_list = list_image_files(seg_data_path, "USER SPECIFIED")

# directory to put the quantification data
out_data_path = data_root_path / "USER SPECIFIED"

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# If use_sample is set to True, then the sample data is used and the directories are set
if use_sample:
    data_root_path, raw_file_list, seg_file_list, raw_data_path, seg_data_path, out_data_path = sample_input_batch()

## **`1` - Locate the matching raw and segmentation files (*`_find_segmentation_tiff_files`*)**


- identify protoype image and segmentations to collect

In [33]:
# linearly unmixed image used as an example
prototype = raw_file_list[0]

# suffix of the corresponding segmentations
test_suffix = "-"

# the segmentations that are to be identified
segs_to_cellect_test = ["lyso","ER","masks"]

- locate folders for raw image and segmentations

In [34]:
# raw
prototype = Path(prototype)
if not prototype.exists():
    print(f"bad prototype. please choose an existing `raw` file as prototype")

out_files = {"raw":prototype}
seg_path = Path(seg_data_path)

# segmentations
if not seg_path.is_dir():
    print(f"bad path argument. please choose an existing path containing organelle segmentations")

- collect all listed region and segmentation filenames

In [35]:
for org_n in segs_to_cellect_test:
    org_name = Path(seg_path) / f"{prototype.stem}{test_suffix}{org_n}.tiff"
    if org_name.exists(): 
        out_files[org_n] = org_name
    else: 
        print(f"{org_n} .tiff file not found in {seg_path} returning")
        out_files[org_n] = None

In [36]:
def _find_segmentation_tiff_files(prototype:Union[Path,str],
                                  name_list:List[str], 
                                  seg_path:Union[Path,str],
                                  suffix:Union[str, None]=None) -> Dict:
    """
    Find the matching segmentation files to the raw image file based on the raw image file path.

    Paramters:
    ---------
    prototype:Union[Path,str]
        the file path (as a string) for one raw image file; this file should have matching segmentation 
        output files with the same file name root and different file name ending that match the strings 
        provided in name_list
    name_list:List[str]
        a list of file name endings related to what segmentation is that file
    seg_path:Union[Path,str]
        the path (as a string) to the matching segmentation files.
    suffix:Union[str, None]=None
        any additional text that exists between the file root and the name_list ending
        Ex) Prototype = "C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/raw/a48hrs-Ctrl_9_Unmixing.czi"
            Name of organelle file = a48hrs-Ctrl_9_Unmixing-20230426_test_cell.tiff
            result of .stem = "a48hrs-Ctrl_9_Unmixing"
            organelle/cell area type = "cell"
            suffix = "-20230426_test_"
    
    Returns:
    ----------
    a dictionary of file paths for each image type (raw and all the different segmentations)

    """
    # raw
    prototype = Path(prototype)
    if not prototype.exists():
        print(f"bad prototype. please choose an existing `raw` file as prototype")
        return dict()

    out_files = {"raw":prototype}
    seg_path = Path(seg_path) 

    # raw
    if not seg_path.is_dir():
        print(f"bad path argument. please choose an existing path containing organelle segmentations")
        return out_files

    # segmentations
    for org_n in name_list:
        org_name = Path(seg_path) / f"{prototype.stem}{suffix}{org_n}.tiff"
        if org_name.exists(): 
            out_files[org_n] = org_name
        elif org_name.exists() == False: 
            org_name = Path(seg_path) / f"{prototype.stem}{suffix}{org_n}.tif"
            out_files[org_n] = org_name
        else: 
            print(f"{org_n} .tiff file not found in {seg_path} returning")
            out_files[org_n] = None
    
    return out_files

In [37]:
prototype = raw_file_list[0]
segs_to_cellect_test = ["LD","ER","golgi","lyso","mito","perox","masks"] #"cell", "cyto", "nuc"]
test_suffix = "-"

filez = _find_segmentation_tiff_files(prototype, segs_to_cellect_test, seg_data_path, test_suffix)

filez

{'raw': WindowsPath('c:/Users/redre/Documents/CohenLab/MSI-3D-analysis/infer-subc/sample_data/batch_example/raw/05052022_astro_control_2_Linear unmixing_0_cmle.ome.tiff'),
 'LD': WindowsPath('c:/Users/redre/Documents/CohenLab/MSI-3D-analysis/infer-subc/sample_data/batch_example/seg/05052022_astro_control_2_Linear unmixing_0_cmle.ome-LD.tiff'),
 'ER': WindowsPath('c:/Users/redre/Documents/CohenLab/MSI-3D-analysis/infer-subc/sample_data/batch_example/seg/05052022_astro_control_2_Linear unmixing_0_cmle.ome-ER.tiff'),
 'golgi': WindowsPath('c:/Users/redre/Documents/CohenLab/MSI-3D-analysis/infer-subc/sample_data/batch_example/seg/05052022_astro_control_2_Linear unmixing_0_cmle.ome-golgi.tiff'),
 'lyso': WindowsPath('c:/Users/redre/Documents/CohenLab/MSI-3D-analysis/infer-subc/sample_data/batch_example/seg/05052022_astro_control_2_Linear unmixing_0_cmle.ome-lyso.tiff'),
 'mito': WindowsPath('c:/Users/redre/Documents/CohenLab/MSI-3D-analysis/infer-subc/sample_data/batch_example/seg/05052022_

In [38]:
from infer_subc.utils.batch import find_segmentation_tiff_files

filez_final = find_segmentation_tiff_files(prototype, segs_to_cellect_test, seg_data_path, test_suffix)

filez==filez_final

True

## **`2` - Establish the parameters for prototype `_batch_process_quantification` function**

> ###### **📝 The lysosome, ER, nucleus and cellmask will serve as examples in the following cells. In the final function, quantification is performed for all listed organelles and regions**

In [39]:
# names of organelles and regions to quantify
organelle_names = ["lyso", "ER"]
region_names = ["nuc","cell"]

# channels for each organelle listed in the order they appear above
organelle_channels = [3,1]

# The file type of the linearly unmixed raw image files 
raw_file_type=".tiff"

# The name of the masks segmentation (can differ based on mask inferring method)
masks_file_name="masks"

# select the cellmask as the masking object for the organelle segmentations
# This is done differently in the function, but is simplified here
mask = "cell"

# Number of bins to be used during calculation of distribution metrics
dist_num_bins=5

# If set to true the bins will be distributed from the center of the centering object to the edge of the cellmask
# If set to false the bins will be distributed from the edge of the centering object to the edge of the cellmask
dist_center_on=False

# Whether or not to include the centering object as the first bin
dist_keep_center_as_bin=True

# the number of zernike degrees to include for the zernike shape descriptors
# dist_zernike_degrees=9
dist_zernike_degrees=None

# The centering object to measure cell distribution from
dist_centering_obj="nuc"

# Whether or not to include distribution data for the interaction sites
include_contact_dist=True

# filename for 
current_time = datetime.now().strftime("%Y%m%d")
out_file_name = f"{current_time}_prototype_"

## **`3` - List raw image files and their corresponding segmentations and to collect**

In [40]:
# reading list of files from the raw path
img_file_list = list_image_files(raw_data_path, raw_file_type)

# Establishing the third image as the prototype
img_f = img_file_list[0]

# list of segmentation files to collect
segs_to_collect = organelle_names + [masks_file_name]
print(segs_to_collect)

['lyso', 'ER', 'masks']


## **`4` - Obtain segmentations and raw image files**

> ###### **📝 The third image in the image file list will serve as the prototype in the following cells. In the final function, quantification is performed for all suitable images (based on file type) in the raw data folder**

- run `find_segmentation_tiff_files` to collect segmentation filenames

In [41]:
filez = find_segmentation_tiff_files(img_f, segs_to_collect, seg_path, test_suffix)

- read in linearly unmixed image file *(raw)*

In [42]:
# read in raw file and metadata
img_data, meta_dict = read_czi_image(filez["raw"])

- collect the listed organelle channel intensities from the raw image file

In [43]:
# create intensities from raw file as list based on the channel order provided
intensities = [img_data[ch] for ch in organelle_channels]

- identify the scale of the raw image from the metadata

In [44]:
# define the scale
# In the finalized function this is an option based on the "scale" boolean parameter
scale_tup = meta_dict['scale']

- read in organelle and region segmentations

In [45]:
# load regions as a list based on order in list (should match order in "masks" file)
masks = read_tiff_image(filez[masks_file_name]) 
regions = [masks[r] for r, region in enumerate(region_names)]

# store organelle images as list
organelles = [read_tiff_image(filez[org]) for org in organelle_names]

## **`5` - Run `make_all_metrics_tables` and store output**

- run `make_all_metrics_tables` for the prototype image

In [46]:
org_metrics, contact_metrics, dist_metrics, region_metrics = make_all_metrics_tables(source_file=img_f,
                                                                                             list_obj_names=organelle_names,
                                                                                             list_obj_segs=organelles,
                                                                                             list_intensity_img=intensities, 
                                                                                             list_region_names=region_names,
                                                                                             list_region_segs=regions, 
                                                                                             mask=mask,
                                                                                             dist_centering_obj=dist_centering_obj,
                                                                                             dist_num_bins=dist_num_bins,
                                                                                             dist_center_on=dist_center_on,
                                                                                             dist_keep_center_as_bin=dist_keep_center_as_bin,
                                                                                             dist_zernike_degrees=dist_zernike_degrees,
                                                                                             scale=scale_tup,
                                                                                             include_contact_dist=include_contact_dist)

It took 3.2394425590833027 minutes to quantify one image.


In [47]:
org_metrics

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,...,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,24,"(0.3961, 0.0799, 0.0799)",3.277839,56.855426,69.993317,2,571,711,...,3.653857,10.312543,0.147918,-147,0.327005,28.333220,464.521332,63768.566406,7695.379883,6348.969238
1,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,159,"(0.3961, 0.0799, 0.0799)",1.219543,38.550233,49.877461,2,479,621,...,14.795673,0.716261,0.452381,1,0.808511,1.141136,560.063293,10399.141602,3539.507812,2599.490234
2,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,173,"(0.3961, 0.0799, 0.0799)",1.621445,42.850571,51.407452,2,525,635,...,8.987179,1.314719,0.215103,1,0.643836,2.840378,1674.058472,19969.951172,5392.369141,3871.452148
3,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,177,"(0.3961, 0.0799, 0.0799)",1.210693,44.246893,55.917647,3,550,696,...,17.491580,0.635173,0.414063,1,0.757143,0.731893,1060.050659,8507.080078,4220.408691,1869.155029
4,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,182,"(0.3961, 0.0799, 0.0799)",1.504691,48.415260,54.300554,3,602,674,...,10.861088,0.944025,0.439394,1,0.820755,1.479506,2134.108154,10107.143555,4662.395020,2259.895996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,714,"(0.3961, 0.0799, 0.0799)",5.941365,61.727967,79.618329,15,771,994,...,35.141437,0.351734,0.750000,1,inf,0.359070,2136.609863,2640.494385,2337.579102,160.849258
210,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,718,"(0.3961, 0.0799, 0.0799)",6.337456,57.321999,78.188166,16,716,977,...,32.871165,0.351734,1.000000,1,inf,0.291925,2182.252441,3363.850586,2696.972412,416.378876
211,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,719,"(0.3961, 0.0799, 0.0799)",6.337456,57.721734,78.987636,16,721,987,...,32.871165,0.351734,1.000000,1,inf,0.291925,2248.788574,3358.940674,2739.195801,310.354767
212,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,720,"(0.3961, 0.0799, 0.0799)",6.460056,58.052943,77.590467,16,723,967,...,15.650987,0.587783,0.437500,1,0.763636,0.859968,1410.446411,5495.817871,3071.461182,1041.830078


In [48]:
contact_metrics

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,...,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length
0,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,291_1,"(0.3961, 0.0799, 0.0799)",1.523427,40.809869,48.835317,3,509,610,...,513,613,1.006807,0.032911,30.591692,0.397602,0.541667,1,0.866667,0.649579
1,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,295_1,"(0.3961, 0.0799, 0.0799)",1.615228,41.870424,49.377136,3,519,613,...,529,623,3.204565,0.194935,16.439137,0.719389,0.256667,1,0.641667,0.999857
2,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,173_1,"(0.3961, 0.0799, 0.0799)",1.862983,42.620870,51.213805,3,525,635,...,545,650,9.093765,0.665817,13.658048,1.083392,0.219167,1,0.552521,1.992485
3,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,24_1,"(0.3961, 0.0799, 0.0799)",3.406541,56.299612,69.536981,3,573,713,...,842,1046,2419.505497,304.637990,7.942232,8.348244,0.095953,-553,0.210602,27.040554
4,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,24_1,"(0.3961, 0.0799, 0.0799)",1.188273,50.753020,62.811693,3,633,784,...,638,788,1.007197,0.030379,33.153850,0.387133,0.600000,1,inf,0.517616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,719_1,"(0.3961, 0.0799, 0.0799)",6.337456,57.721734,78.987636,16,721,987,...,724,990,0.748957,0.022785,32.871165,0.351734,1.000000,1,inf,0.291925
304,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,720_1,"(0.3961, 0.0799, 0.0799)",6.375179,57.992031,77.632344,16,723,968,...,729,975,1.638831,0.053164,30.825878,0.466523,0.250000,1,0.583333,0.777529
305,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,710_1,"(0.3961, 0.0799, 0.0799)",6.337456,58.521204,75.709809,16,732,947,...,733,948,0.180947,0.002532,71.474577,0.169096,1.000000,1,inf,0.000000
306,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,713_1,"(0.3961, 0.0799, 0.0799)",6.348161,59.586444,77.544269,16,742,967,...,750,975,2.711128,0.093670,28.943365,0.563466,0.289062,1,0.521127,0.976748


In [49]:
dist_metrics

,image_name,object,scale,XY_n_bins,XY_bins,XY_mask_vox_cnt_perbin,XY_obj_vox_cnt_perbin,XY_center_vox_cnt_perbin,XY_n_pix_perbin,XY_portion_pix_perbin,...,XY_area_wedges_perbin,Z_n_slices,Z_slices,Z_mask_vox_cnt,Z_obj_vox_cnt,Z_center_vox_cnt,Z_height,Z_mask_volume,Z_obj_volume,Z_center_volume
0,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,"(0.3961, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[217355, 95389, 135234, 192049, 383969]","[16618, 42894, 57510, 66988, 66838]","[79970, 0, 0, 0, 0]","[15534.0, 7146.0, 10646.0, 17048.0, 68991.0]","[0.13013865035814517, 0.05986679512419889, 0.0...",...,"[[13.824863835867001, 7.407774935631001, 7.631...",18,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 7572, 64134, 76764, 81590, 89670, 98309...","[0, 0, 849, 9022, 22719, 26531, 25925, 26496, ...","[0, 0, 0, 0, 0, 0, 0, 9607, 13937, 14701, 1455...",7.129638,"[0.0, 0.0, 19.1694619326348, 162.3632160047015...","[0.0, 0.0, 2.149349337137737, 22.8403176909972...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.3213181..."
1,05052022_astro_control_2_Linear unmixing_0_cml...,ER,"(0.3961, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[217355, 95389, 135234, 192049, 383969]","[51147, 49866, 77546, 111136, 152565]","[79970, 0, 0, 0, 0]","[15534.0, 7146.0, 10646.0, 17048.0, 68991.0]","[0.13013865035814517, 0.05986679512419889, 0.0...",...,"[[13.824863835867001, 7.407774935631001, 7.631...",18,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 7572, 64134, 76764, 81590, 89670, 98309...","[0, 0, 0, 1710, 32772, 54527, 51994, 50164, 48...","[0, 0, 0, 0, 0, 0, 0, 9607, 13937, 14701, 1455...",7.129638,"[0.0, 0.0, 19.1694619326348, 162.3632160047015...","[0.0, 0.0, 0.0, 4.329078170206749, 82.96640338...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.3213181..."
2,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,"(0.3961, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[217355, 95389, 135234, 192049, 383969]","[9082, 20423, 33707, 40647, 28790]","[79970, 0, 0, 0, 0]","[15534.0, 7146.0, 10646.0, 17048.0, 68991.0]","[0.13013865035814517, 0.05986679512419889, 0.0...",...,"[[13.824863835867001, 7.407774935631001, 7.631...",18,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 7572, 64134, 76764, 81590, 89670, 98309...","[0, 0, 0, 134, 6871, 16216, 16423, 15835, 1520...","[0, 0, 0, 0, 0, 0, 0, 9607, 13937, 14701, 1455...",7.129638,"[0.0, 0.0, 19.1694619326348, 162.3632160047015...","[0.0, 0.0, 0.0, 0.339237704565909, 17.39479304...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.3213181..."


In [50]:
region_metrics

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,...,solidity,axis_major_length,min_intensity-lyso_ch,min_intensity-ER_ch,max_intensity-lyso_ch,max_intensity-ER_ch,mean_intensity-lyso_ch,mean_intensity-ER_ch,standard_deviation_intensity-lyso_ch,standard_deviation_intensity-ER_ch
0,05052022_astro_control_2_Linear unmixing_0_cml...,nuc,1,"(0.3961, 0.0799, 0.0799)",3.83653,56.294829,77.344285,7,622,900,...,0.873445,15.282542,0.013644,0.025324,460.053864,2085.241943,5.658617,32.900188,16.937321,100.468475
1,05052022_astro_control_2_Linear unmixing_0_cml...,cell,1,"(0.3961, 0.0799, 0.0799)",3.42331,57.482342,73.509037,2,288,610,...,0.300887,72.743973,0.013644,0.025324,63768.566406,184831.343750,2148.092285,10064.098633,4296.401855,11873.426758


- repeat steps **`3`** and **`4`** and run `make_all_metrics_tables` for all images in `img_file_list`

In [51]:
# containers to collect data tabels
org_tabs = []
contact_tabs = []
dist_tabs = []
region_tabs = []

for img_f in img_file_list:
    filez = find_segmentation_tiff_files(img_f, segs_to_collect, seg_path, test_suffix)

    # read in raw file and metadata
    img_data, meta_dict = read_czi_image(filez["raw"])

    # create intensities from raw file as list based on the channel order provided
    intensities = [img_data[ch] for ch in organelle_channels]

    # define the scale
    # if scale is True:
    if scale:
        scale_tup = meta_dict['scale']
    else:
        scale_tup = None

    # load regions as a list based on order in list (should match order in "masks" file)
    masks = read_tiff_image(filez[masks_file_name]) 
    regions = [masks[r] for r, region in enumerate(region_names)] #TODO: add in option for multiple mask files

    # store organelle images as list
    organelles = [read_tiff_image(filez[org]) for org in organelle_names]

    org_metrics, contact_metrics, dist_metrics, region_metrics = make_all_metrics_tables(source_file=img_f,
                                                                                        list_obj_names=organelle_names,
                                                                                        list_obj_segs=organelles,
                                                                                        list_intensity_img=intensities, 
                                                                                        list_region_names=region_names,
                                                                                        list_region_segs=regions, 
                                                                                        mask=mask,
                                                                                        dist_centering_obj=dist_centering_obj,
                                                                                        dist_num_bins=dist_num_bins,
                                                                                        dist_center_on=dist_center_on,
                                                                                        dist_keep_center_as_bin=dist_keep_center_as_bin,
                                                                                        dist_zernike_degrees=dist_zernike_degrees,
                                                                                        scale=scale_tup,
                                                                                        include_contact_dist=include_contact_dist)
    org_tabs.append(org_metrics)
    contact_tabs.append(contact_metrics)
    dist_tabs.append(dist_metrics)
    region_tabs.append(region_metrics)

It took 2.4936530510584514 minutes to quantify one image.
It took 11.796064209938049 minutes to quantify one image.


In [52]:
read_tiff_image(filez[masks_file_name]) 

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        ...,

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
    

## **`6` - Combine all per image tables into four comprehensive tables**

- batch organelle morphology table

In [53]:
final_org = pd.concat(org_tabs, ignore_index=True)
final_org

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,...,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,24,"(0.3961, 0.0799, 0.0799)",3.277839,56.855426,69.993317,2,571,711,...,3.653857,10.312543,0.147918,-147,0.327005,28.333220,464.521332,63768.566406,7695.379883,6348.969238
1,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,159,"(0.3961, 0.0799, 0.0799)",1.219543,38.550233,49.877461,2,479,621,...,14.795673,0.716261,0.452381,1,0.808511,1.141136,560.063293,10399.141602,3539.507812,2599.490234
2,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,173,"(0.3961, 0.0799, 0.0799)",1.621445,42.850571,51.407452,2,525,635,...,8.987179,1.314719,0.215103,1,0.643836,2.840378,1674.058472,19969.951172,5392.369141,3871.452148
3,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,177,"(0.3961, 0.0799, 0.0799)",1.210693,44.246893,55.917647,3,550,696,...,17.491580,0.635173,0.414063,1,0.757143,0.731893,1060.050659,8507.080078,4220.408691,1869.155029
4,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,182,"(0.3961, 0.0799, 0.0799)",1.504691,48.415260,54.300554,3,602,674,...,10.861088,0.944025,0.439394,1,0.820755,1.479506,2134.108154,10107.143555,4662.395020,2259.895996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,20230727_C2-121_conditioned_well 4_cell 3_untr...,lyso,290,"(0.4106, 0.0799, 0.0799)",10.027746,82.542462,72.218039,23,1030,901,...,15.202034,0.708640,0.493056,1,0.806818,1.366828,264.368591,2812.453369,1094.971924,668.432434
487,20230727_C2-121_conditioned_well 4_cell 3_untr...,lyso,291,"(0.4106, 0.0799, 0.0799)",10.292951,81.855089,72.569077,24,1017,901,...,9.735153,1.096456,0.335459,1,0.782738,1.794237,167.492111,6373.066895,1453.256104,1250.816650
488,20230727_C2-121_conditioned_well 4_cell 3_untr...,lyso,292,"(0.4106, 0.0799, 0.0799)",10.959701,79.030684,73.228377,26,987,914,...,22.568205,0.506987,0.433333,1,0.896552,0.990006,189.703934,1054.725342,590.687439,211.806381
489,20230727_C2-121_conditioned_well 4_cell 3_untr...,lyso,293,"(0.4106, 0.0799, 0.0799)",11.086038,80.506629,71.352697,27,1006,891,...,29.626233,0.391802,1.000000,1,inf,0.399735,431.525635,962.036377,665.161438,158.245163


- batch organelle interactions table

In [54]:
final_contact = pd.concat(contact_tabs, ignore_index=True)
final_contact

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,...,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length
0,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,291_1,"(0.3961, 0.0799, 0.0799)",1.523427,40.809869,48.835317,3,509,610,...,513,613,1.006807,0.032911,30.591692,0.397602,0.541667,1,0.866667,0.649579
1,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,295_1,"(0.3961, 0.0799, 0.0799)",1.615228,41.870424,49.377136,3,519,613,...,529,623,3.204565,0.194935,16.439137,0.719389,0.256667,1,0.641667,0.999857
2,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,173_1,"(0.3961, 0.0799, 0.0799)",1.862983,42.620870,51.213805,3,525,635,...,545,650,9.093765,0.665817,13.658048,1.083392,0.219167,1,0.552521,1.992485
3,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,24_1,"(0.3961, 0.0799, 0.0799)",3.406541,56.299612,69.536981,3,573,713,...,842,1046,2419.505497,304.637990,7.942232,8.348244,0.095953,-553,0.210602,27.040554
4,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,24_1,"(0.3961, 0.0799, 0.0799)",1.188273,50.753020,62.811693,3,633,784,...,638,788,1.007197,0.030379,33.153850,0.387133,0.600000,1,inf,0.517616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,20230727_C2-121_conditioned_well 4_cell 3_untr...,lysoXER,289_1,"(0.4106, 0.0799, 0.0799)",9.443662,82.505304,69.154155,23,1032,865,...,1033,866,0.187442,0.002624,71.425050,0.171135,1.000000,1,inf,0.000000
632,20230727_C2-121_conditioned_well 4_cell 3_untr...,lysoXER,291_1,"(0.4106, 0.0799, 0.0799)",10.253284,81.854468,72.572358,24,1017,901,...,1031,915,6.201975,0.558980,11.095157,1.022035,0.362245,1,0.722034,1.800344
633,20230727_C2-121_conditioned_well 4_cell 3_untr...,lysoXER,290_1,"(0.4106, 0.0799, 0.0799)",10.024691,82.523405,72.213259,24,1030,901,...,1036,907,2.429385,0.139089,17.466407,0.642833,0.736111,1,0.868852,0.915466
634,20230727_C2-121_conditioned_well 4_cell 3_untr...,lysoXER,292_1,"(0.4106, 0.0799, 0.0799)",10.932065,79.037603,73.224790,26,987,914,...,991,919,1.452114,0.062984,23.055393,0.493639,0.600000,1,0.960000,0.889090


- batch distribution measurments table

In [55]:
final_dist = pd.concat(dist_tabs, ignore_index=True)
final_dist

,image_name,object,scale,XY_n_bins,XY_bins,XY_mask_vox_cnt_perbin,XY_obj_vox_cnt_perbin,XY_center_vox_cnt_perbin,XY_n_pix_perbin,XY_portion_pix_perbin,...,XY_area_wedges_perbin,Z_n_slices,Z_slices,Z_mask_vox_cnt,Z_obj_vox_cnt,Z_center_vox_cnt,Z_height,Z_mask_volume,Z_obj_volume,Z_center_volume
0,05052022_astro_control_2_Linear unmixing_0_cml...,lyso,"(0.3961, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[217355, 95389, 135234, 192049, 383969]","[16618, 42894, 57510, 66988, 66838]","[79970, 0, 0, 0, 0]","[15534.0, 7146.0, 10646.0, 17048.0, 68991.0]","[0.13013865035814517, 0.05986679512419889, 0.0...",...,"[[13.824863835867001, 7.407774935631001, 7.631...",18,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 7572, 64134, 76764, 81590, 89670, 98309...","[0, 0, 849, 9022, 22719, 26531, 25925, 26496, ...","[0, 0, 0, 0, 0, 0, 0, 9607, 13937, 14701, 1455...",7.129638,"[0.0, 0.0, 19.1694619326348, 162.3632160047015...","[0.0, 0.0, 2.149349337137737, 22.8403176909972...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.3213181..."
1,05052022_astro_control_2_Linear unmixing_0_cml...,ER,"(0.3961, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[217355, 95389, 135234, 192049, 383969]","[51147, 49866, 77546, 111136, 152565]","[79970, 0, 0, 0, 0]","[15534.0, 7146.0, 10646.0, 17048.0, 68991.0]","[0.13013865035814517, 0.05986679512419889, 0.0...",...,"[[13.824863835867001, 7.407774935631001, 7.631...",18,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 7572, 64134, 76764, 81590, 89670, 98309...","[0, 0, 0, 1710, 32772, 54527, 51994, 50164, 48...","[0, 0, 0, 0, 0, 0, 0, 9607, 13937, 14701, 1455...",7.129638,"[0.0, 0.0, 19.1694619326348, 162.3632160047015...","[0.0, 0.0, 0.0, 4.329078170206749, 82.96640338...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.3213181..."
2,05052022_astro_control_2_Linear unmixing_0_cml...,lysoXER,"(0.3961, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[217355, 95389, 135234, 192049, 383969]","[9082, 20423, 33707, 40647, 28790]","[79970, 0, 0, 0, 0]","[15534.0, 7146.0, 10646.0, 17048.0, 68991.0]","[0.13013865035814517, 0.05986679512419889, 0.0...",...,"[[13.824863835867001, 7.407774935631001, 7.631...",18,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 7572, 64134, 76764, 81590, 89670, 98309...","[0, 0, 0, 134, 6871, 16216, 16423, 15835, 1520...","[0, 0, 0, 0, 0, 0, 0, 9607, 13937, 14701, 1455...",7.129638,"[0.0, 0.0, 19.1694619326348, 162.3632160047015...","[0.0, 0.0, 0.0, 0.339237704565909, 17.39479304...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.3213181..."
3,20230727_C2-121_conditioned_well 4_cell 3_untr...,lyso,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[336257, 150187, 161981, 188126, 1451981]","[5818, 12798, 16480, 16851, 63410]","[138168, 0, 0, 0, 0]","[14616.0, 8329.0, 11910.0, 17166.0, 299375.0]","[0.04159409896526995, 0.023702603330715204, 0....",...,"[[15.729537632949002, 11.249080143840002, 12.1...",34,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 2, 2114, 64537, 170239, 213918, 233313, 22...","[0, 0, 0, 52, 4294, 11265, 15032, 14033, 11771...","[0, 0, 0, 0, 0, 0, 0, 0, 12, 1827, 6983, 9922,...",13.960196,"[0.0, 0.0052486418324770925, 5.547814416928287...","[0.0, 0.0, 0.0, 0.1364646876444044, 11.2688340...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.031..."
4,20230727_C2-121_conditioned_well 4_cell 3_untr...,ER,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[336257, 150187, 161981, 188126, 1451981]","[73178, 83570, 74076, 82379, 396869]","[138168, 0, 0, 0, 0]","[14616.0, 8329.0, 11910.0, 17166.0, 299375.0]","[0.04159409896526995, 0.023702603330715204, 0....",...,"[[15.729537632949002, 11.249080143840002, 12.1...",34,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 2, 2114, 64537, 170239, 213918, 233313, 22...","[0, 0, 12, 1451, 15131, 57457, 102056, 88434, ...","[0, 0, 0, 0, 0, 0, 0, 0, 12, 1827, 6983, 9922,...",13.960196,"[0.0, 0.0052486418324770925, 5.547814416928287...","[0.0, 0.0, 0.031491850994862555, 3.80788964946...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.031..."
5,20230727

- batch cell region morphology table

In [56]:
final_region = pd.concat(region_tabs, ignore_index=True)
final_region

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,...,solidity,axis_major_length,min_intensity-lyso_ch,min_intensity-ER_ch,max_intensity-lyso_ch,max_intensity-ER_ch,mean_intensity-lyso_ch,mean_intensity-ER_ch,standard_deviation_intensity-lyso_ch,standard_deviation_intensity-ER_ch
0,05052022_astro_control_2_Linear unmixing_0_cml...,nuc,1,"(0.3961, 0.0799, 0.0799)",3.836530,56.294829,77.344285,7,622,900,...,0.873445,15.282542,0.013644,0.025324,460.053864,2085.241943,5.658617,32.900188,16.937321,100.468475
1,05052022_astro_control_2_Linear unmixing_0_cml...,cell,1,"(0.3961, 0.0799, 0.0799)",3.423310,57.482342,73.509037,2,288,610,...,0.300887,72.743973,0.013644,0.025324,63768.566406,184831.343750,2148.092285,10064.098633,4296.401855,11873.426758
2,20230727_C2-121_conditioned_well 4_cell 3_untr...,nuc,1,"(0.4106, 0.0799, 0.0799)",6.390325,79.595502,72.097275,8,935,829,...,0.812691,12.490761,0.010097,0.012389,13.215355,102.794571,0.036865,0.904242,0.200762,4.122770
3,20230727_C2-121_conditioned_well 4_cell 3_untr...,cell,1,"(0.4106, 0.0799, 0.0799)",4.152141,72.360158,67.395771,1,0,0,...,0.046408,131.627548,0.010097,0.012389,13269.064453,42737.378906,105.521568,2463.759277,554.816284,4080.432861


## **`7` - Export comprehensive tables as .csv files**

In [57]:
org_csv_path = out_data_path / f"{out_file_name}_organelles.csv"
final_org.to_csv(org_csv_path)

contact_csv_path = out_data_path / f"{out_file_name}_contacts.csv"
final_contact.to_csv(contact_csv_path)

dist_csv_path = out_data_path / f"{out_file_name}_distributions.csv"
final_dist.to_csv(dist_csv_path)

region_csv_path = out_data_path / f"{out_file_name}_regions.csv"
final_region.to_csv(region_csv_path)

## **Define `_batch_process_quantification` function**

In [58]:
# for a list of "prefixes"  collect stats + cross stats masked by cytosol (including nuclei masked by cellmask)

# NOTE: the convex hull regionprops error is a know issue that occurs when the objects being measured have too few voxels. 
# Here's the github reference:https://github.com/scikit-image/scikit-image/issues/5363

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

def _batch_process_quantification(out_file_name: str,
                                  seg_path: Union[Path,str],
                                  out_path: Union[Path, str], 
                                  raw_path: Union[Path,str], 
                                  raw_file_type: str,
                                  organelle_names: List[str],
                                  organelle_channels: List[int],
                                  region_names: List[str],
                                  masks_file_name: str,
                                  mask: str,
                                  dist_centering_obj:str, 
                                  dist_num_bins: int,
                                  dist_center_on: bool=False,
                                  dist_keep_center_as_bin: bool=True,
                                  dist_zernike_degrees: Union[int, None]=None,
                                  include_contact_dist: bool = True,
                                  scale:bool=True,
                                  seg_suffix:Union[str, None]=None) -> int :
    """  
    batch process segmentation quantification (morphology, distribution, contacts); this function is currently optimized to process images from one file folder per image type (e.g., raw, segmentation)
    the output csv files are saved to the indicated out_path folder

    Parameters:
    ----------
    out_file_name: str
        the prefix to use when naming the output datatables
    seg_path: Union[Path,str]
        Path or str to the folder that contains the segmentation tiff files
    out_path: Union[Path, str]
        Path or str to the folder that the output datatables will be saved to
    raw_path: Union[Path,str]
        Path or str to the folder that contains the raw image files
    raw_file_type: str
        the file type of the raw data; ex - ".tiff", ".czi"
    organelle_names: List[str]
        a list of all organelle names that will be analyzed; the names should be the same as the suffix used to name each of the tiff segmentation files
        Note: the intensity measurements collect per region (from get_region_morphology_3D function) will only be from channels associated to these organelles 
    organelle_channels: List[int]
        a list of channel indices associated to respective organelle staining in the raw image; the indices should listed in same order in which the respective segmentation name is listed in organelle_names
    region_names: List[str]
        a list of regions, or masks, to measure; the order should correlate to the order of the channels in the "masks" output segmentation file
    masks_file_name: str
        the suffix of the "masks" segmentation file; ex- "masks_B", "masks", etc.
        this function currently does not accept indivial region segmentations 
    mask: str
        the name of the region to use as the mask when measuring the organelles; this should be one of the names listed in regions list; usually this will be the "cell" mask
    dist_centering_obj:str
        the name of the region or object to use as the centering object in the get_XY_distribution function
    dist_num_bins: int
        the number of bins for the get_XY_distribution function
    dist_center_on: bool=False,
        for get_XY_distribution:
        True = distribute the bins from the center of the centering object
        False = distribute the bins from the edge of the centering object
    dist_keep_center_as_bin: bool=True
        for get_XY_distribution:
        True = include the centering object area when creating the bins
        False = do not include the centering object area when creating the bins
    dist_zernike_degrees: Union[int, None]=None
        for get_XY_distribution:
        the number of zernike degrees to include for the zernike shape descriptors; if None, the zernike measurements will not 
        be included in the output
    include_contact_dist:bool=True
        whether to include the distribution of contact sites in get_contact_metrics_3d(); True = include contact distribution
    scale:bool=True
        a tuple that contains the real world dimensions for each dimension in the image (Z, Y, X)
    seg_suffix:Union[str, None]=None
        any additional text that is included in the segmentation tiff files between the file stem and the segmentation suffix


    Returns:
    ----------
    count: int
        the number of images processed
        
    """
    start = time.time()
    count = 0

    if isinstance(raw_path, str): raw_path = Path(raw_path)
    if isinstance(seg_path, str): seg_path = Path(seg_path)
    if isinstance(out_path, str): out_path = Path(out_path)
    
    if not Path.exists(out_path):
        Path.mkdir(out_path)
        print(f"making {out_path}")
    
    # reading list of files from the raw path
    img_file_list = list_image_files(raw_path, raw_file_type)

    # list of segmentation files to collect
    segs_to_collect = organelle_names + [masks_file_name]

    # containers to collect data tabels
    org_tabs = []
    contact_tabs = []
    dist_tabs = []
    region_tabs = []
    for img_f in img_file_list:
        count = count + 1
        filez = find_segmentation_tiff_files(img_f, segs_to_collect, seg_path, seg_suffix)

        # read in raw file and metadata
        img_data, meta_dict = read_czi_image(filez["raw"])

        # create intensities from raw file as list based on the channel order provided
        intensities = [img_data[ch] for ch in organelle_channels]

        # define the scale
        # if scale is True:
        if scale:
            scale_tup = meta_dict['scale']
        else:
            scale_tup = None

        # load regions as a list based on order in list (should match order in "masks" file)
        masks = read_tiff_image(filez[masks_file_name]) 
        regions = [masks[r] for r, region in enumerate(region_names)] #TODO: add in option for multiple mask files

        # store organelle images as list
        organelles = [read_tiff_image(filez[org]) for org in organelle_names]

        org_metrics, contact_metrics, dist_metrics, region_metrics = make_all_metrics_tables(source_file=img_f,
                                                                                             list_obj_names=organelle_names,
                                                                                             list_obj_segs=organelles,
                                                                                             list_intensity_img=intensities, 
                                                                                             list_region_names=region_names,
                                                                                             list_region_segs=regions, 
                                                                                             mask=mask,
                                                                                             dist_centering_obj=dist_centering_obj,
                                                                                             dist_num_bins=dist_num_bins,
                                                                                             dist_center_on=dist_center_on,
                                                                                             dist_keep_center_as_bin=dist_keep_center_as_bin,
                                                                                             dist_zernike_degrees=dist_zernike_degrees,
                                                                                             scale=scale_tup,
                                                                                             include_contact_dist=include_contact_dist)

        org_tabs.append(org_metrics)
        contact_tabs.append(contact_metrics)
        dist_tabs.append(dist_metrics)
        region_tabs.append(region_metrics)
        end2 = time.time()
        print(f"Completed processing for {count} images in {(end2-start)/60} mins.")

    final_org = pd.concat(org_tabs, ignore_index=True)
    final_contact = pd.concat(contact_tabs, ignore_index=True)
    final_dist = pd.concat(dist_tabs, ignore_index=True)
    final_region = pd.concat(region_tabs, ignore_index=True)

    org_csv_path = out_path / f"{out_file_name}_organelles.csv"
    final_org.to_csv(org_csv_path)

    contact_csv_path = out_path / f"{out_file_name}_contacts.csv"
    final_contact.to_csv(contact_csv_path)

    dist_csv_path = out_path / f"{out_file_name}_distributions.csv"
    final_dist.to_csv(dist_csv_path)

    region_csv_path = out_path / f"{out_file_name}_regions.csv"
    final_region.to_csv(region_csv_path)

    end = time.time()
    print(f"Quantification for {count} files is COMPLETE! Files saved to '{out_path}'.")
    print(f"It took {(end - start)/60} minutes to quantify these files.")
    return count

## **Run `_batch_process_quantification` function**

#### &#x1F6D1; &#x270D; **User Input Required:**

In [13]:
# Whether or not you are using the sample data
use_sample = True

# suffix used for the segmentation files
seg_suffix = "-"

# names of organelles and regions to quantify
organelle_names = ["LD","ER","golgi","lyso","mito","perox"]
region_names = ["nuc", "cell", "cyto"]

# channels for each organelle listed in the order they appear above
organelle_channels = [0,1,2,3,4,5]

# If you are not using the sample data, please edit the below as necessary.

# all the imaging data goes here
data_root_path = Path("USER SPECIFIED")

# linearly unmixed ".czi" files are here
raw_data_path = data_root_path / "USER SPECIFIED"

# adding an additional list of image paths for the matching segmentation files
seg_data_path = data_root_path / "USER SPECIFIED"

# directory to put the quantification data
out_data_path = data_root_path / "USER SPECIFIED"

#### &#x1F3C3; **Run code; no user input required**

In [14]:
if use_sample:
    data_root_path, raw_file_list, seg_file_list, raw_data_path, seg_data_path, out_data_path = sample_input_batch()

In [ ]:
n_files = _batch_process_quantification(out_file_name = "example_prototype",
                                  seg_path=seg_data_path,
                                  out_path=out_data_path, 
                                  raw_path=raw_data_path, 
                                  raw_file_type=".tiff",
                                  organelle_names=organelle_names,
                                  organelle_channels=organelle_channels,
                                  region_names=region_names,
                                  masks_file_name="masks",
                                  mask="cell",
                                  dist_centering_obj="nuc", 
                                  dist_num_bins=5,
                                  dist_center_on=False,
                                  dist_keep_center_as_bin=True,
                                  dist_zernike_degrees=None,
                                  include_contact_dist=True,
                                  scale=True,
                                  seg_suffix=seg_suffix)